In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate
from tensorflow.keras.layers import Reshape


In [2]:
# Carregando o dataset
data = pd.read_csv('C:/Users/mrsilva/Pessoal/Senac/creditcard_2023.csv')

# Contando a quantidade de transações fraudulentas
num_frauds = data['Class'].sum()
print(f"Número de transações fraudulentas: {num_frauds}")

Número de transações fraudulentas: 284315


In [3]:
# Separando features e target
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']

In [4]:
# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Modelo Convolutional Neural Network (CNN)
def build_cnn_model():
    input_layer = Input(shape=(X_train.shape[1],))  
    x = Reshape((X_train.shape[1], 1))(input_layer) 
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# Preparando dados para a CNN com novo input_shape
X_train_cnn = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
# Treinando a CNN com o novo input_shape
cnn_model = build_cnn_model()
cnn_model.fit(X_train_cnn, y_train, epochs=5, batch_size=64, validation_data=(X_test_cnn, y_test))


Epoch 1/5
7108/7108 [==============================] - 35s 5ms/step - loss: 0.0775 - accuracy: 0.9732 - val_loss: 0.0551 - val_accuracy: 0.9816
Epoch 2/5
7108/7108 [==============================] - 23s 3ms/step - loss: 0.0470 - accuracy: 0.9834 - val_loss: 0.0414 - val_accuracy: 0.9853
Epoch 3/5
7108/7108 [==============================] - 38s 5ms/step - loss: 0.0382 - accuracy: 0.9860 - val_loss: 0.0361 - val_accuracy: 0.9880
Epoch 4/5
7108/7108 [==============================] - 33s 5ms/step - loss: 0.0334 - accuracy: 0.9883 - val_loss: 0.0321 - val_accuracy: 0.9900
Epoch 5/5
7108/7108 [==============================] - 30s 4ms/step - loss: 0.0298 - accuracy: 0.9898 - val_loss: 0.0282 - val_accuracy: 0.9919


In [8]:
cnn_model.save('CNN_fraude_cartao.h5')

c:\Users\mrsilva\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# Realizando previsões com a CNN
cnn_predictions = cnn_model.predict(X_test_cnn)
cnn_predictions = np.round(cnn_predictions)

# Avaliando o desempenho da CNN com as novas previsões
cnn_accuracy = accuracy_score(y_test, cnn_predictions)
cnn_confusion = confusion_matrix(y_test, cnn_predictions)
cnn_precision = precision_score(y_test, cnn_predictions)
cnn_recall = recall_score(y_test, cnn_predictions)
cnn_f1 = f1_score(y_test, cnn_predictions)


3554/3554 [==============================] - 5s 1ms/step


In [10]:

print("\n------CNN Metrics:---------\n")
print(f"Accuracy: {cnn_accuracy}\n")
print(f"Confusion Matrix:\n{cnn_confusion}")
print(f"Precision: {cnn_precision}\n")
print(f"Recall: {cnn_recall}\n")
print(f"F1 Score: {cnn_f1}")


------CNN Metrics:---------

Accuracy: 0.9918752088352707

Confusion Matrix:
[[56238   512]
 [  412 56564]]
Precision: 0.9910295045202887

Recall: 0.9927688851446222

F1 Score: 0.9918984322940413
